In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# 작업 디렉토리로 이동(코랩의 input폴더)
import os

os.chdir('/content/gdrive/MyDrive/Colab Notebooks/chatbot2024/')

# 워킹 디렉토리확인
%pwd

# data 폴더 생성 및 디렉토리내 해당 파일 확인
#os.makedirs(data_dir, exist_ok=True)
%ls

In [ ]:
# 패키지 설치
%pip install --upgrade --quiet  streamlit pypdf PyPDF2 langchain langchain-community langchainhub langchain-openai chromadb bs4 faiss-cpu

In [ ]:
! pip install -U sentence-transformers  # 허깅 페이스에서 임베딩 알고리즘

In [ ]:
# 환경변수 준비 - Open AI
import getpass
import os

# 키값 입력
OPENAI_API_KEY = ""

In [ ]:
import bs4
#from langchain import hub
#from loguru import logger

#문서 불러오기/분할/임베딩/저장

# 언어 모델

# 메모리/툴/체인

#from langchain_core.output_parsers import StrOutputParser
#from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings


from langchain.document_loaders import UnstructuredPDFLoader,UnstructuredPowerPointLoader, PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
#from sentence_transformers import SentenceTransformer, util
import numpy as np
import shutil
import os

import webbrowser

import streamlit as st
import tiktoken


from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import UnstructuredPowerPointLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

# from streamlit_chat import message
from langchain.callbacks import get_openai_callback
from langchain.memory import StreamlitChatMessageHistory

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader,UnstructuredPowerPointLoader, PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util
import numpy as np
import shutil
import os

top_k= 2   # 여기서 몇개 가져올건지 결정됨

DATA_PATH='./data/'
DOC_PATH='./docs/'

embedder=SentenceTransformer("jhgan/ko-sroberta-multitask") # 중요

%ls

In [ ]:
# 디렉토리
def get_pdf_files (directory):
  file_list = os.listdir(directory)
  pdf_files = [file for file in file_list if file.endswith('.pdf')]
  return pdf_files

def remove_underscore_after_pdf (pdf_files):
  modified_pdf_files = []

  for file in pdf_files:
    base_name = os.path.splitext(file)[0]
    base_name_without_underscore = base_name.split('_')[0]
    new_file_name = f" {base_name_without_underscore}"
    modified_pdf_files.append(new_file_name)

  return modified_pdf_files

In [ ]:
# 자모깨짐
import unicodedata
def chg_itemname(itemnames):
    itemlists = list()
    for itemname in itemnames:
      itemlists.append(unicodedata.normalize('NFC', itemname))
      # print(unicodedata.normalize('NFC', itemname))
    return itemlists

In [ ]:
#가져온 파일을 doc 폴더에 넣음
def copy_files_to_another_folder (source_folder, target_folder, files_to_copy):#대상 폴더가 없으면 생성
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    for file_to_copy in files_to_copy:
        source_path = os.path.join(source_folder, file_to_copy)
        target_path = os.path.join(target_folder, file_to_copy)

        try:
            shutil.copy2(source_path, target_path)
            print(f"File '{file_to_copy}' copied successfully.")
        except FileNotFoundError:
            print(f"File '{file_to_copy}' not found in the source folder.")

In [ ]:
# 파일 이름 가져오기
pdf_files_full_name = get_pdf_files(DATA_PATH)

pdf_files_name = chg_itemname(remove_underscore_after_pdf (pdf_files_full_name))

filename_embeddings = embedder.encode(pdf_files_name, convert_to_tensor=True)

In [ ]:
# 문서 선택

question = "의료 설명해줘"

In [ ]:
# 파일명을 질문과 유사한 파일명만 가져오기

question_embedding = embedder.encode(question, convert_to_tensor=True)
cos_scores = util.pytorch_cos_sim (question_embedding, filename_embeddings).cpu()

top_results = np.argpartition(-cos_scores, range (top_k)) [0: top_k]
files_to_copy= []

for idx in top_results [0] [0:top_k]:
  files_to_copy.append(pdf_files_full_name [idx])

copy_files_to_another_folder (DATA_PATH, DOC_PATH, files_to_copy)

In [ ]:
loader = PyPDFDirectoryLoader (DOC_PATH)

len (documents)

In [ ]:
loader = PyPDFDirectoryLoader(DOC_PATH)
documents = loader.load()
print(documents)

In [ ]:
documents[1]

In [ ]:
# 파일명 추출
cleaned_documents = []
for document in documents:
  cleaned_document = document #metadata에서 파일명을 추출합니다.
  source_path = document.metadata.get('source', '')
  file_name = source_path.split('docs/')[-1]
  file_name = file_name.split('_')[0]
  file_name = unicodedata.normalize('NFC', file_name)

  page_content=document.page_content

  cleaned_text = f"File Name: {file_name} \n"


# 데이터 클린징..
  #1x00 문자를 제거하고 깨끗한 텍스트를 얻습니다.
  # 특수 문자 제거 .. 한글 형태소 분서, 띄어쓰기..,
  cleaned_text +=page_content.replace("\x00", "") # x00을 공백으로 처리
  cleaned_document.page_content = cleaned_text
  cleaned_documents.append(cleaned_document)

In [ ]:
len(cleaned_documents)

In [ ]:
cleaned_documents[16]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)
all_splits = text_splitter.split_documents(cleaned_documents)

In [ ]:
# 허깅 페이스 기반 임베딩
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(
    model_name = "jhgan/ko-sroberta-multitask",
    encode_kwargs= {'normalize_embeddings': True}
    )

vectorstore = FAISS.from_documents(
#vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings)

In [ ]:
template = """ 당신은 의학에 대해 대답하는 챗봇입니다. 주어진 문서에 있는 내용으로 대답하시오.
만약 문서에 없는 내용이면 문서에 존재하지 않는다고 답변하세요. 모른다고 지어내서 말하지 말하세요.
File Name : 을 참조해서 어떤 절차에 대한 문서인지 파악하고 질문과 동일한 절차의 내용만 사용하세요.

{context}

질문 : {question}
너의 답변 : """

prompt = PromptTemplate(input_variables= ["context", "question"], template=template)

In [ ]:
len(all_splits)

In [ ]:
# 문서 컨텐츠 확인
len(all_splits[0].page_content)